# Export Subset of OpenQuake Results

Sometimes `--export-outputs` produces way too much data, and it's handy to zip up the result.

*Note that this script must be run from a notebook/jupyter environment run from a terminal in a python 2.7 environment. Switching kernels is not enough.*

In [6]:
import os
import sys
import tarfile
import subprocess

In [7]:
# configuration
CALC_ID = 111
export_dir = '../HazardOutputs'
result_dir = os.path.join(export_dir, 'calc_%d' % CALC_ID)
all_outputs_file = os.path.join(result_dir, 'all_outputs.txt')
output_keywords = ['Hazard Curves', 'Hazard Curve (multiple imts)',
                   'Hazard Map', 'UHS']
type_keywords = ['mean', 'quantile']

In [8]:
if not os.path.exists(export_dir):
    os.mkdir(export_dir)
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

In [9]:
# generate listing of available results
command = ['oq-engine', '--list-outputs', str(CALC_ID)]
print('Listing outputs with "%s > %s"' 
      % (' '.join(command), all_outputs_file))
sys.stdout.flush()
with open(all_outputs_file, 'w') as lho_file:
    subprocess.call(command, stdout=lho_file, stderr=subprocess.STDOUT)

Listing outputs with "oq-engine --list-outputs 111 > ../HazardOutputs/calc_111/all_outputs.txt"


In [14]:
# export results of interest
exported_files = []
n_exported, n_results = 0, 0
with open(all_outputs_file, 'r') as lho_file:
    for line in lho_file:
        try:
            result_id = int(line.split()[0])
            n_results += 1
        except ValueError:
            print line.strip()
            continue
        
        is_output = any([key.lower() in line.lower() 
                         for key in output_keywords])
        is_type = any([key.lower() in line.lower() 
                       for key in type_keywords])
        if is_output and is_type:
            print line.strip()
            sys.stdout.flush()
            command = ['oq-engine', '--export-output', 
                       str(result_id), export_dir]
            pipes = subprocess.Popen(command, 
                                     stdout=subprocess.PIPE, 
                                     stderr=subprocess.PIPE)
            std_out, std_err = pipes.communicate()
            if pipes.returncode != 0:
                print
                print('"%s" failed' % (' '.join(command)))
                err_msg = "%s\nReturn Code: %s" % (std_err.strip(), 
                                                   pipes.returncode)
                print(err_msg)
                print
                continue
                
            elif len(std_err):
                print(std_err.strip())
            
            exported_file = std_out.replace('File Exported:','').strip()
            exported_files.append(os.path.relpath(exported_file))

print('Exported %d out of %d results' % (len(exported_files), n_results))

id | output_type | name
17250 | Hazard Curve | 0.01 quantile Hazard Curves PGA

"oq-engine --export-outputs 17250 ../HazardOutputs" failed
Found nothing to export for job 17250
Return Code: 1

17254 | Hazard Curve | 0.01 quantile Hazard Curves SA(0.2)

"oq-engine --export-outputs 17254 ../HazardOutputs" failed
Found nothing to export for job 17254
Return Code: 1

17258 | Hazard Curve | 0.01 quantile Hazard Curves SA(1.0)

"oq-engine --export-outputs 17258 ../HazardOutputs" failed
Found nothing to export for job 17258
Return Code: 1

17251 | Hazard Curve | 0.5 quantile Hazard Curves PGA

"oq-engine --export-outputs 17251 ../HazardOutputs" failed
Found nothing to export for job 17251
Return Code: 1

17255 | Hazard Curve | 0.5 quantile Hazard Curves SA(0.2)

"oq-engine --export-outputs 17255 ../HazardOutputs" failed
Found nothing to export for job 17255
Return Code: 1

17259 | Hazard Curve | 0.5 quantile Hazard Curves SA(1.0)

"oq-engine --export-outputs 17259 ../HazardOutputs" failed
Fou

KeyboardInterrupt: 

In [ ]:
print('Compressing: tar -hzcvf "%s.tar.gz" "%s"' % 
      (result_dir, result_dir))
with tarfile.open(result_dir + '.tar.gz', 'w:gz') as tar:
    tar.add(result_dir, arcname=os.path.split(result_dir)[1])